# Text Analytics Pipeline — **Redacted, Annotated**
This notebook merges multiple scripts into a single, **fully explained** workflow for product-review sentiment & tagging.
All sensitive details are redacted and secrets must be provided via environment variables.


## Pipeline Overview
```
[Ingest/Clean]
     |
     v
[Sentiment Calculation (numeric)] -----> adds: sentiment_score/label
     |
     +--> [LLM Tagger (topics/aspects)] -> adds: tags/aspects, optional refined label
             |
             v
     [Predicting Sentiment (ML model)] -> train/eval + batch inference
             |
             v
     [Breakdown/Reporting] -> per-product/per-time summaries for dashboards
```


## sentiment_calculationNew.py — Purpose, I/O, and How-To

**Purpose**: Compute **numeric sentiment scores** from text/ratings and map to labels.

**Inputs (expected)**: `review_text` (and optionally `star_rating`).

**Outputs**: Adds `sentiment_score` and `sentiment_label` columns.

**When to run**: Run after cleaning to enrich dataset.

**Key functions/classes (auto-extracted)**:
- `def compute_sentiment_proportions(sentences_df)` — Aggregates sentiment counts per review from the sentences sheet.
It extracts topics by building a mapping from the actual predicted sentiment column names.
This approach handles extra spaces and any header discrepancies.

Returns:
  aggregated: dict keyed by review Key. Each value is a dict mapping topic to:
              {"POSITIVE": count, "NEGATIVE": count, "NEUTRAL": count, "count": total, "sentence": example_sentence}
  topics: sorted list of topics identified.
- `def unpivot_reviews(reviews_df, aggregated, topics)` — Builds the unpivoted reviews dataset. Each review from the Reviews sheet is combined
with the aggregated sentiment proportions (from the sentences sheet) on a per-topic basis.

Meta-data from each review is extracted from the Reviews sheet, and dummy store columns
are created. The sentiment proportions for each topic and a representative sentence are added.



In [ ]:
import pandas as pd
from tqdm import tqdm

def compute_sentiment_proportions(sentences_df):
    """
    Aggregates sentiment counts per review from the sentences sheet.
    It extracts topics by building a mapping from the actual predicted sentiment column names.
    This approach handles extra spaces and any header discrepancies.
    
    Returns:
      aggregated: dict keyed by review Key. Each value is a dict mapping topic to:
                  {"POSITIVE": count, "NEGATIVE": count, "NEUTRAL": count, "count": total, "sentence": example_sentence}
      topics: sorted list of topics identified.
    """
    # Define the known prefix and suffix in your sentiment column headers.
    prefix = "Predicted "
    suffix = " Sentiment"
    
    # Build a mapping from topic to its exact sentiment column header.
    sentiment_cols = [
        col for col in sentences_df.columns 
        if col.startswith(prefix) and col.endswith(suffix)
    ]
    topic_to_sentiment_col = {col[len(prefix):-len(suffix)]: col for col in sentiment_cols}
    topics = sorted(topic_to_sentiment_col.keys())
    
    aggregated = {}
    for _, row in tqdm(sentences_df.iterrows(), total=sentences_df.shape[0], desc="Aggregating sentences"):
        review_key = row["Key"]
        if review_key not in aggregated:
            aggregated[review_key] = {}
        
        # Iterate over each topic derived from the mapping.
        for topic in topics:
            # Check if the sentence actually mentions the topic.
            # Adjust this check if the data contains booleans or specific strings.
            topic_val = str(row.get(topic, "")).strip().upper()
            if topic_val != "TRUE":
                continue

            # Use the exact sentiment column name as determined in the mapping.
            sentiment_col = topic_to_sentiment_col.get(topic)
            if not sentiment_col:
                continue

            predicted = row.get(sentiment_col, "")
            if pd.isnull(predicted):
                continue

            predicted = str(predicted).strip().upper()
            if predicted not in ["POSITIVE", "NEGATIVE", "NEUTRAL"]:
                continue

            # Initialize the topic dictionary for the review if necessary.
            if topic not in aggregated[review_key]:
                aggregated[review_key][topic] = {
                    "POSITIVE": 0,
                    "NEGATIVE": 0,
                    "NEUTRAL": 0,
                    "count": 0,
                    "sentence": ""
                }
            aggregated[review_key][topic][predicted] += 1
            aggregated[review_key][topic]["count"] += 1

            # Store the first encountered sentence as a representative example.
            if not aggregated[review_key][topic]["sentence"]:
                aggregated[review_key][topic]["sentence"] = row.get("sentence", "")
                
    return aggregated, topics

def unpivot_reviews(reviews_df, aggregated, topics):
    """
    Builds the unpivoted reviews dataset. Each review from the Reviews sheet is combined
    with the aggregated sentiment proportions (from the sentences sheet) on a per-topic basis.
    
    Meta-data from each review is extracted from the Reviews sheet, and dummy store columns
    are created. The sentiment proportions for each topic and a representative sentence are added.
    """
    unpivoted_rows = []
    
    # Mapping for converting a single "Store" field into dummy columns.
    store_mapping = {
        "Amazon": "Amazon.com",
        "BestBuy": "BestBuy.com",
        "Costco": "Costco.com",
        "Homedepot": "HomeDepot.com",
        "Lowes": "Lowes.com",
        "Walmart": "Walmart.com"
    }
    
    for _, review in reviews_df.iterrows():
        review_key = review["Key"]
        meta = {}
        # Basic meta-data extracted from reviews.
        meta["Key"] = review.get("Key", "")
        meta["Promoted Review"] = review.get("Promoted Review", "")
        meta["product_image"] = review.get("image_list", "")
        # Format the review date to YYYY-MM-DD.
        rev_date = review.get("review_date", None)
        if pd.notnull(rev_date):
            try:
                meta["review_date"] = pd.to_datetime(rev_date, errors="coerce").strftime('%Y-%m-%d')
            except Exception:
                meta["review_date"] = ""
        else:
            meta["review_date"] = ""
        meta["star_rating"] = review.get("star_rating", "")
        meta["review_title"] = review.get("review_title", "")
        meta["review_text"] = review.get("review_text", "")
        meta["Model_number"] = review.get("Model_number", "")
        meta["Brand"] = review.get("Brand", "")
        meta["Price"] = review.get("Price", "")
        
        # Create dummy store columns (set to 1 for matching store, 0 otherwise).
        store_val = str(review.get("Store", "")).strip()
        dummy_stores = {out_key: 0 for out_key in store_mapping.values()}
        for in_store, out_col in store_mapping.items():
            if store_val.lower() == in_store.lower():
                dummy_stores[out_col] = 1
                break
        meta.update(dummy_stores)
        
        # For each topic, add the sentiment proportions.
        for topic in topics:
            new_row = meta.copy()
            new_row["Topic"] = topic
            if review_key in aggregated and topic in aggregated[review_key]:
                data = aggregated[review_key][topic]
                count = data.get("count", 0)
                if count > 0:
                    new_row["Proportion Positive"] = data.get("POSITIVE", 0) / count
                    new_row["Proportion Neutral"] = data.get("NEUTRAL", 0) / count
                    new_row["Proportion Negative"] = data.get("NEGATIVE", 0) / count
                else:
                    new_row["Proportion Positive"] = 0.0
                    new_row["Proportion Neutral"] = 0.0
                    new_row["Proportion Negative"] = 0.0
                new_row["sentence"] = data.get("sentence", "")
            else:
                new_row["Proportion Positive"] = 0.0
                new_row["Proportion Neutral"] = 0.0
                new_row["Proportion Negative"] = 0.0
                new_row["sentence"] = ""
            
            unpivoted_rows.append(new_row)
    
    unpivoted_df = pd.DataFrame(unpivoted_rows)
    return unpivoted_df

if __name__ == "__main__":
    # Load the Reviews and Sentences sheets from the Excel file.
    file_path = "/Users/eureka/Desktop/Predicting_Sentiment Combo.xlsx"
    reviews_df = pd.read_excel(file_path, sheet_name="Reviews")
    sentences_df = pd.read_excel(file_path, sheet_name="Sentences")
    
    # Compute aggregated sentiment counts and sample sentence per review and topic.
    aggregated, topics = compute_sentiment_proportions(sentences_df)
    print("Identified topics:", topics)
    
    # Build the unpivoted reviews dataset.
    unpivoted_df = unpivot_reviews(reviews_df, aggregated, topics)
    
    # Export the unpivoted dataset to an Excel file.
    output_excel = "/Users/eureka/Desktop/Unpivoted Combo Review.xlsx"
    unpivoted_df.to_excel(output_excel, index=False)
    print("Unpivoted reviews dataset saved to:", output_excel)



## GPT-TaggerNew.py — Purpose, I/O, and How-To

**Purpose**: Use an **LLM API** to assign **topics/aspects** and optionally refine sentiment labels.

**Inputs (expected)**: Cleaned `review_text` and metadata; requires API key via env.

**Outputs**: Adds `tags`/`aspects` and possibly `sentiment_label_llm`.

**When to run**: Run post-cleaning. Placeholders are redacted.

**Key functions/classes (auto-extracted)**:
- `def promotedTagger(data_frame)` — Special tag that detects if a review was part of a promotion.
- `def predict_topics_with_gpt(topics, sentence, appliance_type)` — Uses OpenAI service to predict topics for the given sentence.
- `def gpt_tagger_worker(n, q)` — No docstring
- `def main(new_version_of_tag_file, reviews_df, sentences_df_original, output_file, appliance_type_original)` — No docstring



In [ ]:
import pandas as pd
from tqdm import tqdm
import time
from openai import OpenAI
from queue import Queue
from threading import Thread

# Use your API key here – ensure it is valid!
client = OpenAI(api_key="<REDACTED>")

def promotedTagger(data_frame):
    """Special tag that detects if a review was part of a promotion."""
    print("\nTagging for the Promoted Review tag.")
    print("Searching for the following phrase:")
    print("'review was collected as part of a promotion'")
    promoted_keys = []
    result = []
    for index, row in tqdm(data_frame.iterrows(), total=data_frame.shape[0]):
        row_tag = False
        # Check if the review text contains the target phrase (case-insensitive)
        if "review was collected as part of a promotion" in str(row['review_text']).lower():
            row_tag = True
            promoted_keys.append(row['Key'])
        result.append(row_tag)
    return {"Promoted Review Bool List": result, "Promoted Keys": promoted_keys}

def predict_topics_with_gpt(topics, sentence, appliance_type):
    """
    Uses OpenAI service to predict topics for the given sentence.
    """
    model_name = "gpt-4o"
    output = "Not Available"
    
    retries = 5
    print("-" * 80)
    print(sentence)
    while retries > 0:
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "user",
                     "content": f"In the context of {appliance_type} online reviews, does the following sentence talk about any of the following topics {str(topics)}? Return only a list of the relevant topics from my list and nothing else. Sentence: {sentence}"}
                ],
                max_tokens=50,
                temperature=0,
                timeout=20
            )
            output = response.choices[0].message.content
            print("\n" + output + "\n")
            break
        except Exception as e:
            retries -= 1
            if retries > 0:
                time.sleep(3)
            else:
                print("Wasn't able to detect the relevant topics. Assigning 'Not Available'\n")
                print(e)
    return output, model_name

def gpt_tagger_worker(n, q):
    global topics_only_list
    global sentences_df
    global appliance_type
    
    while True:
        try:
            # Try to get an item from the queue.
            data = q.get(block=False, timeout=1)
        except Exception as e:
            # When queue is empty, the thread will exit.
            print("Thread", n, "has joined")
            break
        
        try:
            index = data[0]
            row = data[1]
            if len(str(row["sentence"])) > 2:
                if "review was collected as part of a promotion" in str(row["sentence"]).lower():
                    # If it is a promoted sentence, skip GPT tagging.
                    gpts_response_for_topics = ""
                else:
                    gpts_response_for_topics, _ = predict_topics_with_gpt(topics_only_list, str(row["sentence"]), appliance_type)
            else:
                print("-" * 80)
                print(str(row["sentence"]), "not long enough to predict topics")
                gpts_response_for_topics = ""
            
            # NEW FIX:
            # If the GPT call failed (returned "Not Available"), mark this sentence as untagged and continue.
            if gpts_response_for_topics == "Not Available":
                sentences_df.at[index, "Untagged"] = True
                q.task_done()
                continue
            
            # Sort topics by length to avoid partial matching issues.
            sorted_topics_only_list = sorted(topics_only_list, key=len, reverse=True)
            topics_gpt_found = []
            for topic in sorted_topics_only_list:
                if topic in gpts_response_for_topics:
                    sentences_df.at[index, topic] = True
                    topics_gpt_found.append(topic)
                    # Remove the matched topic from the response to avoid duplicate tagging.
                    gpts_response_for_topics = gpts_response_for_topics.replace(topic, "")
            
            if topics_gpt_found == []:
                sentences_df.at[index, "Untagged"] = True

            print("Great Success")
            q.task_done()
        except Exception as e:
            print(e)
            print("Couldn't process data:", data)
            q.task_done()

# Global variables used by threads
things_to_tag_q = Queue(maxsize=0)
topics_only_list = []
sentences_df = pd.DataFrame()
appliance_type = ""
CONCURRENCY = 30

def main(new_version_of_tag_file, reviews_df, sentences_df_original, output_file, appliance_type_original):
    global things_to_tag_q, topics_only_list, sentences_df, appliance_type
    things_to_tag_q = Queue(maxsize=0)
    topics_only_list = []
    sentences_df = sentences_df_original
    appliance_type = appliance_type_original

    try:
        # Read the tag file – the Excel file headers correspond to the topics.
        tag_topic_and_context_df = pd.read_excel(new_version_of_tag_file)
        topics_only_list = tag_topic_and_context_df.columns.tolist()
        
        # Tag reviews that are part of a promotion.
        promoted_tagger_dict = promotedTagger(reviews_df)
        reviews_df["Promoted Review"] = promoted_tagger_dict["Promoted Review Bool List"]
        sentences_df["Promoted Review"] = False
        
        # Tag sentences from promoted reviews.
        for index, row in sentences_df.iterrows():
            if row["Key"] in promoted_tagger_dict["Promoted Keys"]:
                sentences_df.at[index, "Promoted Review"] = True
        
        # Create new columns for each topic and for 'Untagged' in both dataframes.
        for topic in topics_only_list + ["Untagged"]:
            sentences_df[topic] = False
            reviews_df[topic] = False
        
        # Enqueue each sentence row for GPT tagging.
        for index, row in tqdm(sentences_df.iterrows(), total=sentences_df.shape[0]):
            things_to_tag_q.put([index, row])
            
        threads = []
        for i in range(CONCURRENCY):
            thread = Thread(target=gpt_tagger_worker, args=(i, things_to_tag_q))
            threads.append(thread)
            thread.start()
        
        for thread in threads:
            thread.join()
            
        # Apply sentence-level topic tags to reviews by matching the Keys.
        for index, row in reviews_df.iterrows():
            matching_sentences = sentences_df[sentences_df["Key"] == row["Key"]]
            for topic in topics_only_list + ["Untagged"]:
                if True in matching_sentences[topic].tolist():
                    reviews_df.at[index, topic] = True
                    
        # Export the updated data to an Excel file with two sheets.
        output_file = output_file.split(".xlsx")[0] + ".xlsx"
        print("Exporting to:", output_file)
        writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
        writer.book.strings_to_urls = False
        sentences_df.to_excel(writer, sheet_name='Sentences', index=False)
        reviews_df.to_excel(writer, sheet_name='Reviews', index=False)
        writer._save()
    
    except KeyboardInterrupt:
        backup_file = output_file.split(".xlsx")[0] + " Backup.xlsx"
        print("KeyboardInterrupt detected. Exporting to backup file:", backup_file)
        writer = pd.ExcelWriter(backup_file, engine='xlsxwriter')
        writer.book.strings_to_urls = False
        sentences_df.to_excel(writer, sheet_name='Sentences', index=False)
        reviews_df.to_excel(writer, sheet_name='Reviews', index=False)
        writer._save()
    
    return reviews_df, sentences_df

if __name__ == '__main__':
    tag_file = input("Enter name of tag file: ")
    reviews_file_name = input("Enter name of reviews df file: ")
    reviews_sheet_name = input("Enter name of reviews sheet: ")
    sentences_file_name = input("Enter name of sentences df file: ")
    sentences_sheet_name = input("Enter name of sentences sheet: ")
    output_file = input("Enter name of output file: ")
    appliance_type_original = input("Enter the appliance type: ")
    
    main(
        tag_file,
        pd.read_excel(reviews_file_name, sheet_name=reviews_sheet_name),
        pd.read_excel(sentences_file_name, sheet_name=sentences_sheet_name),
        output_file,
        appliance_type_original
    )


## predicting_sentimentNew.py — Purpose, I/O, and How-To

**Purpose**: Train/evaluate a **ML classifier** for sentiment; supports batch inference.

**Inputs (expected)**: Text + ground-truth label for training/validation.

**Outputs**: Metrics (F1/precision/recall), confusion matrix, predictions, model artifact.

**When to run**: Run for baseline/production classifier or alongside LLM.

**Key functions/classes (auto-extracted)**:
- **class PredictSentiment** — No docstring
  - `def predict_with_gpt(self, sentence, topic)` — Uses OpenAI service to predict sentiment.
- `def excelSheetToDf(fileName, sheetName)` — input: string fileName: name of excel file ending in .xlsx
       string sheetName: name of the sheet you want to pull from fileName
output: pandas dataframe df: uses the first row in the excel file as headers
- `def predict_sentiment_worker(n, q)` — No docstring
- `def main(reviews_df, sentences_df, output_file, topics_original)` — No docstring



In [ ]:
""" For every tagged sentence, this program makes a prediction whether
it is positive, neutral or negative sentiment using only GPT.
It then matches sentences to reviews with the same key (Unique Identifier)
Josh Phillips, RAC co-op, Spring 2021 """

import pandas as pd
from tqdm import tqdm
import time

from queue import Queue
from threading import Thread
from openai import OpenAI

# UPDATED: Directly create the OpenAI client using your API key.
client = OpenAI(api_key="<REDACTED>")

class PredictSentiment:
    def predict_with_gpt(self, sentence, topic):
        """
        Uses OpenAI service to predict sentiment.
        """
        retries = 5
        output = "NOT AVAILABLE"
        while retries > 0:
            try:
                if topic != "Noise":
                    response = client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=[
                            {"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user",
                             "content": f"Analyze the following product review for topic: {topic} and determine if the sentiment regarding the topic is: POSITIVE, NEGATIVE or NEUTRAL. Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL: {sentence}"}
                        ],
                        max_tokens=10,
                        temperature=0,
                        timeout=10)
                else:
                    print(str(sentence))
                    response = client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=[
                            {"role": "system", "content": "You are a helpful assistant."},
                            {"role": "user",
                             "content": f"Analyze the following product review for topic: {topic} and determine if the sentiment regarding the topic is: POSITIVE, NEGATIVE or NEUTRAL. Quiet is POSITIVE and Loud or unusual noises is NEGATIVE. Return only a single word, either POSITIVE, NEGATIVE or NEUTRAL: {sentence}"}
                        ],
                        max_tokens=10,
                        temperature=0,
                        timeout=10)
                output = response.choices[0].message.content
                print("\n" + output + "\n")
                break
            except Exception as e:
                retries -= 1
                if retries > 0:
                    time.sleep(3)
                else:
                    print("Wasn't able to detect sentiment. Assigning Not Available\n")
                    print(e)
        return output.upper()

# Global variables
things_to_predict_q = Queue(maxsize=0)
CONCURRENCY = 50
topics = None
sentiment_predictor = None
data_frame = None

def excelSheetToDf(fileName, sheetName):
    """
    input: string fileName: name of excel file ending in .xlsx
           string sheetName: name of the sheet you want to pull from fileName
    output: pandas dataframe df: uses the first row in the excel file as headers
    """
    excel_file = pd.ExcelFile(fileName)
    df = excel_file.parse(sheetName)
    return df

def predict_sentiment_worker(n, q):
    global topics, sentiment_predictor, data_frame
    while True:
        try:
            data = q.get(block=False, timeout=1)
            index = data[0]
            row = data[1]

            # Identify which topics are mentioned in the sentence (assumes a column per topic with a truthy value)
            topics_mentioned_in_sentence = []
            for topic in topics:
                if row[topic]:
                    topics_mentioned_in_sentence.append(topic)

            # For each mentioned topic, predict sentiment using GPT.
            for topic in topics_mentioned_in_sentence:
                # Check for promotion-related text; if found, assign NEUTRAL
                if "this review was collected as part of a promotion" in str(row['sentence']).lower():
                    sentiment = "NEUTRAL"
                else:
                    sentiment = sentiment_predictor.predict_with_gpt(row['sentence'], topic)
                data_frame.loc[index, 'Predicted ' + topic + ' Sentiment'] = sentiment

            print("Great Success")
            q.task_done()

        except Exception as e:
            print("Thread", n, "has joined")
            break

def main(reviews_df, sentences_df, output_file, topics_original):
    global things_to_predict_q, topics, sentiment_predictor, data_frame

    things_to_predict_q = Queue(maxsize=0)
    topics = topics_original

    # Create the predictor using GPT only.
    sentiment_predictor = PredictSentiment()
    data_frame = sentences_df

    print("Loading the dataframe")
    # Create a new prediction column for each topic.
    for topic in topics:
        data_frame['Predicted ' + topic + ' Sentiment'] = 'Not Mentioned'

    print("Loading up the queue of the sentences")
    for index, row in tqdm(data_frame.iterrows(), total=data_frame.shape[0]):
        things_to_predict_q.put([index, row])

    threads = []
    for i in range(CONCURRENCY):
        threads += [Thread(target=predict_sentiment_worker, args=(i, things_to_predict_q))]

    for thread in threads:
        thread.start()

    for thread in threads:
        thread.join()

    # Matching the Sentences to the Reviews
    print("Loading the entire reviews into a dataframe")
    review_data_frame = reviews_df

    # Convert boolean columns in reviews to integers.
    a_list_of_cols_to_be_int = []
    for col in review_data_frame.columns:
        if review_data_frame[col].dtype == bool:
            a_list_of_cols_to_be_int.append(col)
        print(a_list_of_cols_to_be_int)

    for col in a_list_of_cols_to_be_int:
        review_data_frame[col] = review_data_frame[col].astype(int)

    print("Matching Sentences with Reviews")
    for index, topic in tqdm(enumerate(topics), total=len(topics)):
        print("-" * 100)
        print(topic)
        review_data_frame['Negatively Mentions ' + topic] = 0
        review_data_frame['Neutrally Mentions ' + topic] = 0
        review_data_frame['Positively Mentions ' + topic] = 0

        print(data_frame[data_frame[topic] == 1])
        for idx, row in data_frame[data_frame[topic] == 1].iterrows():
            sentiment = row['Predicted ' + topic + ' Sentiment']
            if sentiment == "NEGATIVE":
                review_data_frame.loc[review_data_frame.Key == row['Key'], 'Negatively Mentions ' + topic] = 1
            elif sentiment == "NEUTRAL":
                review_data_frame.loc[review_data_frame.Key == row['Key'], 'Neutrally Mentions ' + topic] = 1
            elif sentiment == "POSITIVE":
                review_data_frame.loc[review_data_frame.Key == row['Key'], 'Positively Mentions ' + topic] = 1
            else:
                print("*" * 100)
                print(sentiment)

    print("Exporting to:", output_file)
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    writer.book.strings_to_urls = False

    data_frame.to_excel(writer, sheet_name='Sentences', index=False)
    review_data_frame.to_excel(writer, sheet_name='Reviews', index=False)
    writer._save()

    return review_data_frame, data_frame

if __name__ == '__main__':
    # Prompt for the input files.
    previous_file_name = input("Enter the name of the previous file (with Reviews and Sentences sheets): ")
    output = input("Enter base name of output file (e.g., output.xlsx): ")
    topics_file = input("Enter the name of the topics file (xlsx) with topics in the first row: ")

    # Read topics from the first row of the provided Excel file.
    topics_df = pd.read_excel(topics_file, header=None)
    topics_original = topics_df.iloc[0].tolist()

    reviews_df = pd.read_excel(previous_file_name, sheet_name="Reviews")
    sentences_df = pd.read_excel(previous_file_name, sheet_name="Sentences")
    main(reviews_df, sentences_df, output, topics_original)


## SentimentBreakdownNew.py — Purpose, I/O, and How-To

**Purpose**: Aggregate labeled reviews to produce **sentiment/topic breakdowns** by product and time (counts, proportions, trends).

**Inputs (expected)**: Reviews with: `review_id`, `product_id`, `created_at`, and `sentiment`/`topic` columns.

**Outputs**: `sentiment_breakdown.csv` or DataFrame with grouped stats.

**When to run**: Run after labeling steps to generate reports/dashboards.

**Key functions/classes (auto-extracted)**:
- `def read_input_file(file_name, sheet_name)` — Reads the input file. If the file is an Excel file (.xlsx), it uses the specified sheet name.
If it's a CSV, it loads it directly.
- `def filterDataFrame(df, column_name)` — Ensures the specified column is of string type and fills NaN values with empty strings.
- `def lemmatise(sentence, stop_words)` — Tokenizes the sentence, removes punctuation and stop words, then lemmatizes the remaining words.
- `def main(starting_df, output_file)` — No docstring



In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm import tqdm
import re
import sys

# Download required NLTK resources.
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('punkt')

# Initialize the sentiment analyzer.
sid = SentimentIntensityAnalyzer()

def read_input_file(file_name, sheet_name=None):
    """
    Reads the input file. If the file is an Excel file (.xlsx), it uses the specified sheet name.
    If it's a CSV, it loads it directly.
    """
    if file_name.endswith('.xlsx'):
        excel_file = pd.ExcelFile(file_name)
        df = excel_file.parse(sheet_name)
    elif file_name.endswith('.csv'):
        df = pd.read_csv(file_name)
    else:
        raise ValueError("Unsupported file type. Please use .xlsx or .csv.")
    return df

def filterDataFrame(df, column_name):
    """
    Ensures the specified column is of string type and fills NaN values with empty strings.
    """
    df[column_name] = df[column_name].astype(str)
    df.fillna("", inplace=True)
    return df

def lemmatise(sentence, stop_words):
    """
    Tokenizes the sentence, removes punctuation and stop words, then lemmatizes the remaining words.
    """
    words = nltk.word_tokenize(sentence)
    # Remove apostrophes, convert to lowercase, and keep alphabetic words only.
    words = [word.replace("'", "") for word in words]
    words = [word.lower() for word in words if word.isalpha()]
    lem = WordNetLemmatizer()
    words = [lem.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

def main(starting_df, output_file):
    # --- KEY COLUMN HANDLING (CHANGED) ---
    # If a unique key column "Key" is not present, create one.
    if 'Key' not in starting_df.columns:
        if 'id_x' in starting_df.columns:
            starting_df['Key'] = starting_df['id_x']
        else:
            starting_df['Key'] = starting_df.index

    starting_df = filterDataFrame(starting_df, "review_text")
    
    # Compute overall sentiment for each review.
    overall_sentiments = []
    for index, review in tqdm(enumerate(starting_df['review_text']), total=starting_df.shape[0], desc="Calculating overall sentiment"):
        overall_sentiments.append(sid.polarity_scores(str(review))['compound'])
    starting_df["review_sentiment_score"] = overall_sentiments

    # Prepare stop words.
    stop_words = set(stopwords.words("english"))
    # Remove common negations from stop words since they are important for sentiment.
    negation = ["mustn't", "aren't", "ain", "mightn", "needn", "wasn", "shan't",
                "hadn", "mightn't", "isn't", "hasn", "shan", "hadn't", "shouldn", "needn't",
                "doesn", "haven", "no", "wasn't", "mustn", "haven't", "didn", "weren't",
                "wouldn't", "don't", "couldn't", "weren", "nor", "aren", "didn't", "wouldn",
                "isn", "hasn't", "couldn", "don", "doesn't", "won't", "off", "on", "shouldn't", "not",
                "does", "did"]
    for word in negation:
        if word in stop_words:
            stop_words.remove(word)
    # Add additional words to ignore.
    promoted = ['review', 'collected', 'part', 'promotion', 'rating', 'provided', 'verified', 'purchaser']
    for word in promoted:
        stop_words.add(word)
        
    # Prepare lists to store sentence-level data.
    keys = []
    sentences_only = []
    lemmas = []
    sentence_sentiments = []
    row_list = []
    
    print("Performing sentiment analysis on sentences...")
    # Process each review in the dataset.
    for index, row in tqdm(starting_df.iterrows(), total=starting_df.shape[0], desc="Breaking reviews into sentences"):
        review_text = str(row['review_text'])
        # --- CHANGED CODE START ---
        # Use regex to insert a space after punctuation only when not followed by another punctuation.
        review_text = re.sub(r'([.!?])(?=[^\s.!?])', r'\1 ', review_text)
        # --- CHANGED CODE END ---
        
        # Tokenize into sentences.
        sentences = nltk.sent_tokenize(review_text)
        for sentence in sentences:
            # --- KEY COLUMN HANDLING (CHANGED) ---
            # Instead of using row['id'], we use row['Key'] which we ensured exists.
            keys.append(row['Key'])
            sentences_only.append(sentence)
            lemmas.append(lemmatise(sentence, stop_words))
            sentence_sentiments.append(sid.polarity_scores(sentence)['compound'])
            row_list.append(row.to_dict())
    
    # Create a DataFrame for the sentence-level breakdown.
    sentences_df = pd.DataFrame(row_list)
    # Insert new columns for sentence, lemma, and sentence sentiment score after the "review_text" column.
    review_text_col_index = starting_df.columns.get_loc("review_text")
    sentences_df.insert(loc=review_text_col_index+1, column="sentence", value=sentences_only)
    sentences_df.insert(loc=review_text_col_index+2, column="lemma", value=lemmas)
    sentences_df.insert(loc=review_text_col_index+3, column="<REDACTED>", value=sentence_sentiments)
    
    # Export the results to an Excel file with two sheets: one for Reviews and one for Sentences.
    print("Exporting results to:", output_file)
    writer = pd.ExcelWriter(output_file, engine='xlsxwriter')
    writer.book.strings_to_urls = False
    starting_df.to_excel(writer, sheet_name='Reviews', index=False)
    sentences_df.to_excel(writer, sheet_name='Sentences', index=False)
    writer._save()
    
    return starting_df, sentences_df

if __name__ == '__main__':
    file_name = input("Enter the name of the file containing the starting data frame (xlsx or csv): ")
    output_file = input("Enter the name of the output Excel file to create (e.g., output.xlsx): ")
    
    # If an Excel file is provided, ask for the sheet name; otherwise, for CSV, skip it.
    if file_name.endswith('.xlsx'):
        sheet_name = input("Enter the sheet name: ")
        df = read_input_file(file_name, sheet_name)
    elif file_name.endswith('.csv'):
        df = read_input_file(file_name)
    else:
        print("Unsupported file type. Please provide a .csv or .xlsx file.")
        sys.exit(1)
        
    main(df, output_file)
